In [1]:
!nvidia-smi

Mon Jul 29 07:43:34 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   52C    P0             14W /   56W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
!nvidia-smi

Mon Jul 29 07:43:54 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   57C    P0             14W /   72W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# %pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [5]:
# %pip install --upgrade accelerate
# %pip uninstall -y transformers accelerate
# %pip install transformers accelerate

In [6]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [7]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
from datasets import load_metric, load_dataset
import matplotlib.pyplot as plt
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")


e:\Anaconda\envs\textsum\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from box import ConfigBox

In [9]:
## Setting up the device
## Code taken from: https://huggingface.co/google/pegasus-cnn_dailymail?library=transformers
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
## Loading the samsum data
dataset = load_dataset("samsum", trust_remote_code=True)
type(dataset)

datasets.dataset_dict.DatasetDict

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [12]:
# train_data = pd.DataFrame(dataset.train)
# test_data = pd.DataFrame(dataset.test)
# validation_data = pd.DataFrame(dataset.validation)

In [13]:
# validation_data.head()

In [14]:
## More info here: https://huggingface.co/docs/transformers/main/en/model_doc/pegasus#transformers.PegasusForConditionalGeneration
from ensure import ensure_annotations
@ensure_annotations
def convert_examples_features(example_batch, 
                              input_encoding_length:int = 32, 
                              target_encoding_length: int = 8):
    input_encodings = tokenizer(text=example_batch["dialogue"], 
                                max_length=input_encoding_length, 
                                truncation = True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(text = example_batch["summary"], 
                                     max_length = target_encoding_length, 
                                     truncation=True, padding="max_length")

    return {"input_ids": input_encodings["input_ids"],
                      "attention_mask": input_encodings["attention_mask"],
                      "labels":target_encodings["input_ids"]}

In [15]:
dataset_pt = dataset.map(convert_examples_features, batched = True)
dataset_pt

Map:   0%|          | 0/818 [00:00<?, ? examples/s]e:\Anaconda\envs\textsum\lib\site-packages\transformers\tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 2067.40 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [16]:
dataset_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [17]:
## Datacollator helps to batch the data while feeding it to the model
from transformers import DataCollatorForSeq2Seq
seq2seqdatacollator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)


In [18]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(output_dir="pegasus-samsum", 
                                 num_train_epochs=1, 
                                 warmup_steps=500,
                                 per_device_train_batch_size=1,
                                 per_device_eval_batch_size=1, 
                                 weight_decay=0.01,
                                 logging_steps=10, evaluation_strategy="steps",
                                 eval_steps=500,save_steps=1e6,
                                 gradient_accumulation_steps=16,
                                 gradient_checkpointing=True,
                                 fp16=True #Enable mixed precision training
                                 )


e:\Anaconda\envs\textsum\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# We were unable to train the model due to OOM Error. So, reducing the size of dataset

small_test_data = dataset_pt["test"].select(range(25))
small_val_data = dataset_pt["validation"].select(range(5))

dataset_pt["test"] = small_test_data
dataset_pt["validation"] = small_val_data

dataset_pt["test"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 25
})

In [20]:
trainer = Trainer(model=model_pegasus, 
                  args=trainer_args, 
                  data_collator=seq2seqdatacollator,
                  train_dataset=dataset_pt["test"],
                  eval_dataset=dataset_pt["validation"])

e:\Anaconda\envs\textsum\lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [21]:
trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]e:\Anaconda\envs\textsum\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
e:\Anaconda\envs\textsum\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
100%|██████████| 1/1 [00:49<00:00, 49.25s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (htt

{'train_runtime': 81.7019, 'train_samples_per_second': 0.306, 'train_steps_per_second': 0.012, 'train_loss': 5.310182571411133, 'epoch': 0.64}


TrainOutput(global_step=1, training_loss=5.310182571411133, metrics={'train_runtime': 81.7019, 'train_samples_per_second': 0.306, 'train_steps_per_second': 0.012, 'total_flos': 1444732207104.0, 'train_loss': 5.310182571411133, 'epoch': 0.64})

In [22]:
## Model Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    Split the dataset into smaller batches so that we can process the evaluation simultaneously
    """

    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:batch_size+i]

def calculate_metric_on_eval_ds(dataset, 
                                metric, 
                                model, 
                                batch_size, 
                                device, 
                                column_text = "dialogue", 
                                column_summary = "summary"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text],batch_size=batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset["summary"], batch_size=batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, 
                           max_length=32, 
                           truncation = True, 
                           padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids = inputs["input_ids"].to(device),
                                   attention_mask = inputs["attention_mask"].to(device),
                                   length_penalty = 0.8, num_beams = 8, max_length = 128)
        
        ## Decoding the summary
        decoded_summaries = [tokenizer.decode(s,
                                              skip_special_tokens=True, 
                                              clean_up_tokenization_spaces=True) for s in summaries]
        
        
        metric.add_batch(predictions = decoded_summaries, references = target_batch)

    print("Dialogue\n",article_batch,"\n")
    print("Predicted Summary\n", decoded_summaries)
    score = metric.compute()
    return score

In [23]:
rouge_metric = load_metric("rouge", trust_remote_code=True)

C:\Users\ACER\AppData\Local\Temp\ipykernel_18900\1438680113.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge", trust_remote_code=True)


In [24]:
score = calculate_metric_on_eval_ds(small_val_data, 
                                    rouge_metric, trainer.model, 1, device)
score

  0%|          | 0/5 [00:00<?, ?it/s]e:\Anaconda\envs\textsum\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
e:\Anaconda\envs\textsum\lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
100%|██████████| 5/5 [00:31<00:00,  6.39s/it]


Dialogue
 ['Robert: Hey give me the address of this music shop you mentioned before\r\nRobert: I have to buy guitar cable\r\nFred: <file_other>\r\nFred: Catch it on google maps\r\nRobert: thx m8\r\nFred: ur welcome'] 

Predicted Summary
 ['Fred: I have to buy cable.<n>Robert: I have to buy guitar cable.<n>Fred: I have to buy a guitar.']


{'rouge1': AggregateScore(low=Score(precision=0.08653846153846154, recall=0.09168035030103995, fmeasure=0.08125874125874126), mid=Score(precision=0.21099067599067595, recall=0.22527151211361743, fmeasure=0.21456592188299506), high=Score(precision=0.3615384615384615, recall=0.3808270676691729, fmeasure=0.37317073170731707)),
 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.10476190476190475, recall=0.10952380952380951, fmeasure=0.10695970695970695), high=Score(precision=0.22857142857142856, recall=0.22857142857142856, fmeasure=0.22857142857142856)),
 'rougeL': AggregateScore(low=Score(precision=0.059743589743589745, recall=0.07788724685276409, fmeasure=0.06478900409934893), mid=Score(precision=0.19428904428904428, recall=0.21321262927433526, fmeasure=0.19709516723814452), high=Score(precision=0.3628787878787879, recall=0.3703007518796993, fmeasure=0.36622889305816136)),
 'rougeLsum': AggregateScore(low=Score(precision=0.0597435897435897

In [25]:
rouge_measures = ['rouge1','rouge2','rougeL','rougeLsum']

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_measures)
rouge_dict

{'rouge1': 0.21456592188299506,
 'rouge2': 0.10695970695970695,
 'rougeL': 0.19709516723814452,
 'rougeLsum': 0.20087803701765014}

In [27]:
pd.DataFrame(rouge_dict, index = ["Pegasus"])

,rouge1,rouge2,rougeL,rougeLsum
Pegasus,0.214566,0.10696,0.197095,0.200878


In [28]:
torch.cuda.empty_cache()

In [29]:
## Save the model
model_pegasus.save_pretrained("pegasus-samsum")

## Save the tokenizer
tokenizer.save_pretrained("tokenizer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\spiece.model',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [39]:
## Prediction
sample_text = dataset["train"][0]["dialogue"]
reference_text = dataset["train"][0]["summary"]
## Define the pipeline
pipe = pipeline("summarization", model="pegasus-samsum", tokenizer="tokenizer")
pipe

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [40]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

In [43]:
print("Dialogue: \n")
print(sample_text,"\n")
print("Actual Summary: \n")
print(reference_text,"\n")
print("Generated Summary: \n")
print(pipe(sample_text, num_workers=-1, **gen_kwargs)[0]["summary_text"].replace("<n>", ""))

Your max_length is set to 128, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


Dialogue: 

Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-) 

Actual Summary: 

Amanda baked cookies and will bring Jerry some tomorrow. 

Generated Summary: 

Amanda: I'll bring you tomorrow :-)Amanda: I baked cookies. Do you want some?Amanda: I'll bring you tomorrow :-)
